In [ ]:
# Importing the libraries

import os
import random
import pandas as pd
!pip install deepface
from deepface import DeepFace


In [ ]:
data_directory = "/content/drive/Othercomputers/My Laptop/Quang/Data Science/Python/Pandas/facesData/train"
output_filename = "stress_nostress_differences.csv"

In [ ]:
# Initialize lists to store the results
file_paths = []  # List to store image file paths
stress_numbers = []  # List to store stress image numbers
nostress_numbers = []  # List to store nostress image numbers
stress_level = []  # List to store stress level
nostress_level = []  # List to store nostress level
differences = []  # List to store stress level differences

In [ ]:
# Initialize a set to keep track of used pairs
used_pairs = set()

In [ ]:
# Calculate the number of images in the "stress" and "nostress" folders
stress_folder = os.path.join(data_directory, "stress")
nostress_folder = os.path.join(data_directory, "nostress")

num_stress_images = len([f for f in os.listdir(stress_folder) if os.path.isfile(os.path.join(stress_folder, f))])
num_nostress_images = len([f for f in os.listdir(nostress_folder) if os.path.isfile(os.path.join(nostress_folder, f))])

In [ ]:
# Run the loop for 1500 instances
for _ in range(1500):
    # Generate random numbers for image selection
    stress_number = random.randint(1, num_stress_images)  # Random stress image number
    nostress_number = random.randint(1, num_nostress_images)  # Random nostress image number

    # Create image file paths based on random numbers
    stress_image_path = os.path.join(data_directory, "stress", f"stress_no_{stress_number}.jpg")
    nostress_image_path = os.path.join(data_directory, "nostress", f"nostress_no_{nostress_number}.jpg")

    # Check if this pair has already been used, if so, generate new random numbers
    while (stress_number, nostress_number) in used_pairs:
        stress_number = random.randint(1, num_stress_images)
        nostress_number = random.randint(1, num_nostress_images)
        stress_image_path = os.path.join(data_directory, "stress", f"stress_no_{stress_number}.jpg")
        nostress_image_path = os.path.join(data_directory, "nostress", f"nostress_no_{nostress_number}.jpg")

    # Add this pair to the set of used pairs
    used_pairs.add((stress_number, nostress_number))

    # Analyze the images using DeepFace
    stress_predictions = DeepFace.analyze(stress_image_path, enforce_detection=False)
    nostress_predictions = DeepFace.analyze(nostress_image_path, enforce_detection=False)

    # Calculate stress level for each image
    stress_stress_level = (stress_predictions[0]['emotion']['fear'] + stress_predictions[0]['emotion']['sad'] +
                           stress_predictions[0]['emotion']['angry'] + stress_predictions[0]['emotion']['disgust']) / \
                          (stress_predictions[0]['emotion']['happy'] + stress_predictions[0]['emotion']['neutral'])

    nostress_stress_level = (nostress_predictions[0]['emotion']['fear'] + nostress_predictions[0]['emotion']['sad'] +
                            nostress_predictions[0]['emotion']['angry'] + nostress_predictions[0]['emotion']['disgust']) / \
                           (nostress_predictions[0]['emotion']['happy'] + nostress_predictions[0]['emotion']['neutral'])

    # Calculate the difference between the stress levels
    stress_nostress_difference = stress_stress_level - nostress_stress_level

    # Store the results in lists
    file_paths.append((stress_image_path, nostress_image_path))
    stress_numbers.append(stress_number)
    nostress_numbers.append(nostress_number)
    stress_level.append(stress_stress_level)
    nostress_level.append(nostress_stress_level)
    differences.append(stress_nostress_difference)

In [ ]:
# Create a DataFrame and save it as a CSV file
data = pd.DataFrame({'File Path': file_paths, 'Stress Number': stress_numbers, 'Nostress Number': nostress_numbers, 'Stress Level': stress_level, 'Nostress Level': nostress_level, 'Difference': differences})
data.to_csv(output_filename, index=False)